[用之前将Notebook置于项目根目录下运行]

# 1 准备数据集

以BloodMNIST数据集为例。

准备好数据集，将数据集数据装入Dataloader中用于批量特征提取。

并准备好以下变量：
1. `label_list`: 标签文本列表，如BloodMNIST的标签文本列表为`['basophil', 'eosinophil', ...]`
2. `label_prompt_template_list`：用于标签的提示词模板列表。样例：
    ```python
    label_prompt_template_list = [
    "a microscopic image of a {} cell",
    "a peripheral blood smear image of a {}",
    "a bloodcell of {}"
    ]
    ```
3. `concept_list`: 概念文本列表，如BloodMNIST的概念文本列表可为`['Segmented nucleus','Band nucleus (band form)','Reniform / indented nucleus','Round nucleus', ...]`
4. `concept_prompt_template_list`：用于概念的提示词模板列表。样例：
    ```python
    concept_prompt_template_list = [
    "a cell photo with sign of {}",
    "a photo of a cell with {}",
    "a cell image indicating {}",
    ]
    ```
5. `train_loader`, `test_loader`, `valid_loader`: 分别为训练集、测试集、验证集的Dataloader (torch.utils.data.DataLoader)。

In [1]:
import torch
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms

import numpy as np

from tqdm import tqdm

import medmnist
from medmnist import BloodMNIST, INFO

In [2]:
# 选择BloodMNIST数据集的信息
data_flag = 'bloodmnist'
info = INFO[data_flag]

In [3]:
# 标签映射字典 label_map
label_map = {int(k): v for k, v in info['label'].items()}  # 如 0:'basophil', 1:'eosinophil', ...
label_map[3] = 'immature granulocytes'

# 创建标签文本列表
label_list = [label_map[idx] for idx in sorted(label_map.keys())]
label_list

['basophil',
 'eosinophil',
 'erythroblast',
 'immature granulocytes',
 'lymphocyte',
 'monocyte',
 'neutrophil',
 'platelet']

In [ ]:
# 选择15个细胞特征作为概念（手动选择）
cell_features = [
    {"en": "Segmented nucleus", "zh": "分叶细胞核"},
    {"en": "Band nucleus (band form)", "zh": "带状细胞核（未分叶）"},
    {"en": "Reniform / indented nucleus", "zh": "肾形/凹陷细胞核"},
    {"en": "Round nucleus", "zh": "圆形细胞核"},
    {"en": "Fine azurophilic granules", "zh": "细嗜天青颗粒"},
    {"en": "Eosinophilic granules", "zh": "嗜酸性颗粒（橙红）"},
    {"en": "Basophilic granules", "zh": "嗜碱性颗粒（深紫粗颗粒）"},
    {"en": "Basophilic cytoplasm", "zh": "嗜碱性胞质"},
    {"en": "Cytoplasmic vacuoles", "zh": "胞质空泡"},
    {"en": "High nuclear-to-cytoplasmic ratio", "zh": "高核浆比"},
    {"en": "Pale cytoplasm", "zh": "淡染胞质"},
    {"en": "Nucleated erythrocyte (erythroblast)", "zh": "有核红细胞（幼红细胞）"},
    {"en": "Platelet fragments / clumps", "zh": "血小板碎片/成团"},
    {"en": "Stain precipitate (artifact)", "zh": "染色沉淀（伪影）"},
    {"en": "Overlapping cell clumps (artifact)", "zh": "细胞重叠/成团（伪影）"}
]

# 生成概念文本列表
concept_list = [cell_feature["en"] for cell_feature in cell_features]

In [ ]:
# 准备Prompt模板
concept_prompt_template_list = ["a cell photo with sign of {}",
                   "a photo of a cell with {}",
                   "a cell image indicating {}",
                   "an image of a cell showing {}",
                   "blood cell with {}",
                   "a blood cell photo with sign of {}",
                   "a photo of a blood cell with {}",
                   "a blood cell image indicating {}",
                   "an image of blood cell showing {}"
                   ]

label_prompt_template_list = ["a microscopic image of a {} cell",
                         "a peripheral blood smear image of a {}",
                         "a bloodcell of {}"
                         ]

DB_METADATA = {
    "label_texts": label_list,
    "prompt_temp_for_labels": label_prompt_template_list,
    "concept_texts": concept_list,
    "prompt_temp_for_concepts": concept_prompt_template_list,
}

In [6]:
DataClass = getattr(medmnist, info['python_class'])  # 通过DataClass类获取BloodMNIST子类

common_tf = transforms.Compose([
    # Converts a PIL Image or numpy.ndarray (H x W x C) in the range[0, 255] 
    # to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]
    transforms.ToTensor(),])

ds_train = DataClass(split='train', size=224, as_rgb=True, mmap_mode='r', transform=common_tf, download=True)
ds_test = DataClass(split='test', size=224, as_rgb=True, mmap_mode='r', transform=common_tf, download=True)
ds_valid = DataClass(split='val', size=224, as_rgb=True, mmap_mode='r', transform=common_tf, download=True)

print("num_train:", len(ds_train), "sample shape:", np.array(ds_train[0][0]).shape)  # (224, 224, 3)
print("num_test:", len(ds_test), "sample shape:", np.array(ds_test[0][0]).shape)  # (224, 224, 3)
print("num_valid:", len(ds_valid), "sample shape:", np.array(ds_valid[0][0]).shape)  # (224, 224, 3)

Using downloaded and verified file: C:\Users\chenk\.medmnist\bloodmnist_224.npz
Using downloaded and verified file: C:\Users\chenk\.medmnist\bloodmnist_224.npz
Using downloaded and verified file: C:\Users\chenk\.medmnist\bloodmnist_224.npz
num_train: 11959 sample shape: (3, 224, 224)
num_test: 3421 sample shape: (3, 224, 224)
num_valid: 1712 sample shape: (3, 224, 224)


In [9]:
# 创建Dataloader
train_loader = DataLoader(ds_train, batch_size=64, shuffle=False)
test_loader = DataLoader(ds_test, batch_size=64, shuffle=False)
valid_loader = DataLoader(ds_valid, batch_size=64, shuffle=False)

# 2 载入CLIP模型

In [ ]:
# 载入HuggingFace上的CLIP模型
from transformers import AutoModel, AutoProcessor
from huggingface_hub import login, whoami
login(token="<YOUR_HF_TOKEN>")  # 替换为你的HF token（生产环境中请勿硬编码）
print(whoami())

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\chenk\.cache\huggingface\token
Login successful
{'type': 'user', 'id': '6693342048f8a55b7eba3d8e', 'name': 'takedachia', 'fullname': 'Jack Chan', 'email': 'chenkai1989129@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/df59d16febde1a73fc98bf70f5476ba2.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'ConceptCLIP', 'role': 'read', 'createdAt': '2025-08-20T17:55:27.543Z'}}}


In [13]:
# 载入模型和处理器，第一次会下载模型到本地缓存目录
model = AutoModel.from_pretrained('JerrryNie/ConceptCLIP', trust_remote_code=True)
processor = AutoProcessor.from_pretrained('JerrryNie/ConceptCLIP', trust_remote_code=True)

c:\Users\chenk\.conda\envs\pyg_monai\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\chenk\.conda\envs\pyg_monai\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


In [14]:
# 将模型载入GPU（如没有则用CPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

In [ ]:
# 更改processor的图像预处理设置
# BloodMNIST数据集的DataLoader已经是[0,1]范围内的浮点数，所以关闭processor中的rescale，开启normalize
processor.image_processor.do_rescale = False
processor.image_processor.do_normalize = True

# 3 使用文本和图像编码器进行推理并提取、存储特征（Reason with Text and Image Encoder）

## (1) 部署H5数据库用于存储输出（Deploy h5 for storing outputs）

In [25]:
import os, h5py, hashlib, json, time
from pathlib import Path
from typing import Optional, Sequence

OUT_PATH = "./conceptclip_features.h5"
D      = 1152
T_img  = 729
DTYPE_DYNAMIC = "float16"   # dynamic parts (image_*) can save space
DTYPE_TEXT    = "float32"   # Text features are recommended to keep higher precision
DTYPE_STR     = h5py.string_dtype(encoding="utf-8")

def _serialize_for_attr(value):
    """Serialize a value for storing in HDF5 attribute"""
    return json.dumps(value, ensure_ascii=False)

def _read_json_attr(f: h5py.File, key: str, default=None):
    """
    从 H5 文件根属性("attrs")中读取 JSON 格式的属性值并解析。
    """
    if key not in f.attrs:
        return default
    raw = f.attrs[key]
    if isinstance(raw, bytes):
        raw = raw.decode("utf-8")
    if isinstance(raw, str):
        try:
            return json.loads(raw)
        except json.JSONDecodeError:
            return raw
    return raw

def _hash_texts(texts: Sequence[str]) -> str:
    """Compute a hash value for a list of texts"""
    h = hashlib.sha256()
    for t in texts:
        h.update(t.encode("utf-8")); h.update(b"\0")
    return h.hexdigest()

def init_file(path, d=D, t_img=T_img, metadata: Optional[dict] = None):
    """Init HDF5 file structure"""
    if not os.path.exists(path):
        with h5py.File(path, "w") as f:
            # Create main datasets
            f.create_dataset("image_features",
                shape=(0, d), maxshape=(None, d),
                chunks=(64, d), dtype=DTYPE_DYNAMIC, compression="lzf")
            f.create_dataset("image_token_features",
                shape=(0, t_img, d), maxshape=(None, t_img, d),
                chunks=(4, t_img, d), dtype=DTYPE_DYNAMIC, compression="lzf")
            f.create_dataset("ids",
                shape=(0,), maxshape=(None,),
                chunks=(4096,), dtype="int64", compression="lzf")
            f.create_dataset("labels",
                shape=(0,), maxshape=(None,),
                chunks=(4096,), dtype="int64", compression="lzf")
            f.create_dataset("split",
                shape=(0,), maxshape=(None,),
                chunks=(4096,), dtype=DTYPE_STR, compression="lzf")
            
            # Create prompt-templates group
            f.create_group("templates")
            
            # Add global attributes (model parameters)
            f.attrs["D"] = d
            f.attrs["T_img"] = t_img
            f.attrs["created_at"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            f.attrs["version"] = "1.0"
            
    f = h5py.File(path, "a")
    current_len = f["image_features"].shape[0] if "image_features" in f else 0
    if "image_token_features" not in f:
        f.create_dataset("image_token_features",
            shape=(current_len, t_img, d), maxshape=(None, t_img, d),
            chunks=(4, t_img, d), dtype=DTYPE_DYNAMIC, compression="lzf")
    if "ids" not in f:
        f.create_dataset("ids",
            shape=(current_len,), maxshape=(None,),
            chunks=(4096,), dtype="int64", compression="lzf")
    if "labels" not in f:
        ds_labels = f.create_dataset("labels",
            shape=(current_len,), maxshape=(None,),
            chunks=(4096,), dtype="int64", compression="lzf")
        if current_len > 0:
            ds_labels[:] = np.full((current_len,), -1, dtype="int64")
    if "split" not in f:
        ds_split = f.create_dataset("split",
            shape=(current_len,), maxshape=(None,),
            chunks=(4096,), dtype=DTYPE_STR, compression="lzf")
        if current_len > 0:
            ds_split[:] = np.array(["unspecified"] * current_len, dtype=object)
    if "templates" not in f:
        f.create_group("templates")
    if metadata:
        for key, value in metadata.items():
            try:
                f.attrs[key] = _serialize_for_attr(value)
            except TypeError:
                f.attrs[key] = _serialize_for_attr(str(value))
    return f

def _to_np(x, dtype):
    """Convert tensor to numpy array, with specified dtype"""
    if torch.is_tensor(x):
        x = x.detach().cpu().numpy()
    return x.astype(dtype, copy=False)

def append_batch(f: h5py.File,
                 image_feats: torch.Tensor,            # [B, D]
                 image_token_feats: torch.Tensor,      # [B, T_img, D]
                 ids: np.ndarray,                     # [B] int64
                 split_names: Optional[Sequence[str]] = None,
                 labels: Optional[np.ndarray] = None):
    """Append a batch of data to the HDF5 file"""
    ds_img = f["image_features"]
    ds_tok = f["image_token_features"]
    ds_ids = f["ids"]
    ds_labels = f["labels"]
    ds_split = f["split"]
    B = image_feats.shape[0]

    n0 = ds_img.shape[0]
    ds_img.resize(n0 + B, axis=0)
    ds_tok.resize(n0 + B, axis=0)
    ds_ids.resize(n0 + B, axis=0)
    ds_labels.resize(n0 + B, axis=0)
    ds_split.resize(n0 + B, axis=0)

    ds_img[n0:n0+B, :]      = _to_np(image_feats, DTYPE_DYNAMIC)
    ds_tok[n0:n0+B, :, :]   = _to_np(image_token_feats, DTYPE_DYNAMIC)
    ds_ids[n0:n0+B]         = ids.astype("int64", copy=False)

    if labels is None:
        labels_arr = np.full((B,), -1, dtype="int64")
    else:
        labels_arr = np.asarray(labels, dtype="int64").reshape(-1)
        if labels_arr.shape[0] != B:
            raise ValueError(f"Label count {labels_arr.shape[0]} does not match batch size {B}.")
    ds_labels[n0:n0+B] = labels_arr

    if split_names is None:
        split_arr = np.array(["unspecified"] * B, dtype=object)
    else:
        split_arr = np.asarray(list(split_names), dtype=object).reshape(-1)
        if split_arr.shape[0] != B:
            raise ValueError(f"Split count {split_arr.shape[0]} does not match batch size {B}.")
    ds_split[n0:n0+B] = split_arr

    f.flush()

def write_template(f: h5py.File,
                   template_id: str,
                   texts: Sequence[str],
                   text_features: torch.Tensor,            # [K, D]
                   text_token_features: Optional[torch.Tensor]=None # [K, T_txt, D]
                   ):
    """Write text templates and features"""
    g_root = f["templates"]
    if template_id in g_root:
        del g_root[template_id]             # Overwrite (or change to skip after verification)
    g = g_root.create_group(template_id)

    # Main data
    tf  = _to_np(text_features, DTYPE_TEXT)         # [K, D]
    g.create_dataset("text_features", data=tf, compression="lzf")

    if text_token_features is not None:
        ttf = _to_np(text_token_features, DTYPE_TEXT)     # [K, T_txt, D]
        g.create_dataset("text_token_features", data=ttf, compression="lzf")
        T_txt = ttf.shape[1]
    else:
        T_txt = -1

    # Save original prompts (variable-length UTF-8 strings)
    dt_str = h5py.string_dtype(encoding='utf-8')
    ds_txt = g.create_dataset("texts", shape=(len(texts),), dtype=dt_str, compression="lzf")
    ds_txt[:] = np.array(list(texts), dtype=object)

    # Metadata
    g.attrs["K"]          = tf.shape[0]
    g.attrs["D"]          = tf.shape[1]
    g.attrs["T_txt"]      = int(T_txt)
    g.attrs["texts_hash"] = _hash_texts(texts)
    g.attrs["created_at"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    f.flush()

def save_model_params(f: h5py.File, 
                      logit_scale: float = None,
                      logit_bias: float = None,
                      concept_logit_scale: float = None,
                      concept_logit_bias: float = None):
    """Save key model parameters to the root attributes of the HDF5 file"""
    if logit_scale is not None:
        f.attrs["logit_scale"] = float(logit_scale)
    if logit_bias is not None:
        f.attrs["logit_bias"] = float(logit_bias)
    if concept_logit_scale is not None:
        f.attrs["concept_logit_scale"] = float(concept_logit_scale)
    if concept_logit_bias is not None:
        f.attrs["concept_logit_bias"] = float(concept_logit_bias)
    f.flush()

def load_model_params(f: h5py.File) -> dict:
    """Load model parameters from HDF5 file"""
    params = {}
    for key in ["logit_scale", "logit_bias", "concept_logit_scale", "concept_logit_bias"]:
        if key in f.attrs:
            params[key] = f.attrs[key]
    return params

## (2) 使用文本编码器计算并保存概念文本特征到H5（Compute and save concept text features to H5, using the Text Encoder）

In [26]:
def _maybe_to_scalar(x):
    if x is None:
        return None
    if isinstance(x, (int, float)):
        return float(x)
    if torch.is_tensor(x):
        x = x.detach()
        if x.numel() == 1:
            return float(x.item())
        try:
            return float(x.squeeze().item())
        except Exception:
            return None
    if hasattr(x, "item"):
        try:
            return float(x.item())
        except Exception:
            return None
    return None

stored_templates = []
with init_file(OUT_PATH, metadata=DB_METADATA) as f:
    concept_texts_in_file = _read_json_attr(f, "concept_texts", concept_list)
    concept_template_list = _read_json_attr(f, "prompt_temp_for_concepts", concept_prompt_template_list)
    label_texts_in_file = _read_json_attr(f, "label_texts", label_list)
    label_template_list = _read_json_attr(f, "prompt_temp_for_labels", label_prompt_template_list)

    template_texts_map = {}
    if isinstance(concept_template_list, (list, tuple)) and concept_texts_in_file:
        for idx, template in enumerate(concept_template_list, start=1):
            formatted_texts = [template.format(text) for text in concept_texts_in_file]
            template_key = f"concept_prompts_t{idx:02d}"
            template_texts_map[template_key] = formatted_texts
    if isinstance(label_template_list, (list, tuple)) and label_texts_in_file:
        for idx, template in enumerate(label_template_list, start=1):
            formatted_label_texts = [template.format(text) for text in label_texts_in_file]
            template_key = f"label_prompts_t{idx:02d}"
            template_texts_map[template_key] = formatted_label_texts

    with torch.no_grad():
        for template_id, template_texts in template_texts_map.items():
            text_inputs = processor(text=template_texts, return_tensors="pt", padding=True, truncation=True).to(device)
            text_cls, text_tokens = model.encode_text(text_inputs["input_ids"], normalize=True)  # [K, D], [K, T_txt, D]
            text_tokens_proj = model.text_proj(text_tokens) if hasattr(model, "text_proj") else text_tokens
            write_template(f, template_id=template_id, texts=template_texts, text_features=text_cls, text_token_features=text_tokens_proj)
            stored_templates.append((template_id, text_cls.shape[0]))
    params_kwargs = {
        "logit_scale": _maybe_to_scalar(getattr(model, "logit_scale", None)),
        "logit_bias": _maybe_to_scalar(getattr(model, "logit_bias", None)),
        "concept_logit_scale": _maybe_to_scalar(getattr(model, "concept_logit_scale", None)),
        "concept_logit_bias": _maybe_to_scalar(getattr(model, "concept_logit_bias", None)),
    }
    save_model_params(f, **{k: v for k, v in params_kwargs.items() if v is not None})

print("Stored templates:")
for template_id, count in stored_templates:
    print(f"  - {template_id}: {count} prompts")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Stored templates:
  - concept_prompts_t01: 15 prompts
  - concept_prompts_t02: 15 prompts
  - concept_prompts_t03: 15 prompts
  - concept_prompts_t04: 15 prompts
  - concept_prompts_t05: 15 prompts
  - concept_prompts_t06: 15 prompts
  - concept_prompts_t07: 15 prompts
  - concept_prompts_t08: 15 prompts
  - concept_prompts_t09: 15 prompts
  - label_prompts_t01: 8 prompts
  - label_prompts_t02: 8 prompts
  - label_prompts_t03: 8 prompts


## (3) 使用图像编码器计算并保存图像（token）特征到H5（Compute and save image (token) features to H5, using the Image Encoder）
token即各27×27的patches

In [27]:
# Using Image Encoder, compute and save image (token) features to H5

# Define data splits and their loaders
split_loaders = [
    ("train", train_loader),
    ("val", valid_loader),
    ("test", test_loader),
]

def encode_and_store_image_features(split_loaders):
    """Traverse each data split and write image features to HDF5"""
    written_counts = {}
    with torch.no_grad():
        with init_file(OUT_PATH, metadata=DB_METADATA) as f:
            existing_counts = {}
            if "split_counts" in f.attrs:
                try:
                    existing_counts = json.loads(f.attrs["split_counts"])
                except Exception:
                    existing_counts = {}
            start_idx = f["image_features"].shape[0]
            for split_name, loader in split_loaders:
                expected = len(loader.dataset)
                recorded = int(existing_counts.get(split_name, 0) or 0)
                if recorded >= expected:
                    print(f"Skip {split_name}: already stored {recorded}/{expected} samples.")
                    continue

                processed_in_split = 0
                for imgs, labels in tqdm(loader, desc=f"{split_name} split"):
                    batch_total = imgs.shape[0]
                    if processed_in_split + batch_total <= recorded:
                        processed_in_split += batch_total
                        continue
                    if processed_in_split < recorded:
                        offset = recorded - processed_in_split
                        imgs = imgs[offset:]
                        labels = labels[offset:]
                        processed_in_split = recorded
                        batch_total = imgs.shape[0]

                    pixel_inputs = processor(images=imgs, return_tensors="pt", padding=True, truncation=True)
                    pixel_inputs = pixel_inputs["pixel_values"].to(device)
                    img_cls, img_tokens = model.encode_image(pixel_inputs, normalize=True)   # [B,1152], [B,729,1152]
                    img_tokens_proj = model.image_proj(img_tokens) if hasattr(model, "image_proj") else img_tokens  #  [B,729,1152]
                    batch_size = img_cls.shape[0]
                    batch_ids = np.arange(start_idx, start_idx + batch_size, dtype=np.int64)
                    labels_tensor = labels.view(-1) if hasattr(labels, "view") else labels
                    labels_np = np.asarray(labels_tensor, dtype=np.int64).reshape(-1)
                    append_batch(
                        f,
                        image_feats=img_cls,
                        image_token_feats=img_tokens_proj,
                        ids=batch_ids,
                        split_names=[split_name] * batch_size,
                        labels=labels_np,
                    )
                    start_idx += batch_size
                    processed_in_split += batch_size
                    written_counts[split_name] = written_counts.get(split_name, 0) + batch_size

            # Update split counts in file attributes
            for split_name, count in written_counts.items():
                existing_counts[split_name] = int(existing_counts.get(split_name, 0) or 0) + count
            f.attrs["split_counts"] = json.dumps(existing_counts)
            f.flush()

    total_new = sum(written_counts.values())
    if total_new == 0:
        print("No new samples were appended.")
    else:
        print(f"Appended {total_new} samples to {OUT_PATH}.")
        for split_name, count in written_counts.items():
            print(f"  - {split_name}: {count}")

encode_and_store_image_features(split_loaders)

Skip train: already stored 11959/11959 samples.
Skip val: already stored 1712/1712 samples.
Skip test: already stored 3421/3421 samples.
No new samples were appended.


## (4) 验证数据库内容（Validate H5 contents）

In [28]:
def validate_h5(path: str = OUT_PATH, max_templates: int = 2):
    if not os.path.exists(path):
        print(f"File '{path}' does not exist yet.")
        return
    with h5py.File(path, "r") as f:
        num_samples = f["image_features"].shape[0]
        print(f"Total samples stored: {num_samples}")
        print(f"image_features dataset shape: {f['image_features'].shape}")
        print(f"image_token_features dataset shape: {f['image_token_features'].shape}")
        ids_preview = f["ids"][:5]
        labels_preview = f["labels"][:5]
        split_preview = f["split"][:5]
        print(f"Preview ids: {ids_preview}")
        print(f"Preview labels: {labels_preview}")
        print(f"Preview splits: {split_preview}")
        try:
            split_counts = json.loads(f.attrs.get("split_counts", "{}"))
        except Exception:
            split_counts = {}
        print(f"Recorded split counts: {split_counts}")
        metadata_keys = [
            "label_texts",
            "prompt_temp_for_labels",
            "concept_texts",
            "prompt_temp_for_concepts",
        ]
        for key in metadata_keys:
            value = _read_json_attr(f, key, None)
            if value is None:
                continue
            if isinstance(value, list):
                preview = value[:3] + (["..."] if len(value) > 3 else [])
                print(f"{key}: {preview}")
            else:
                print(f"{key}: {value}")
        templates = list(f["templates"].keys())
        print(f"Templates stored: {templates}")
        for template_id in templates[:max_templates]:
            g = f["templates"][template_id]
            texts_sample = g["texts"][:3] if "texts" in g else []
            print(f"  • Template '{template_id}' -> K={g.attrs.get('K')}, D={g.attrs.get('D')}, sample texts={texts_sample}")
        params = load_model_params(f)
        print(f"Stored model params: {params}")

validate_h5()

Total samples stored: 17092
image_features dataset shape: (17092, 1152)
image_token_features dataset shape: (17092, 729, 1152)
Preview ids: [0 1 2 3 4]
Preview labels: [7 3 6 6 7]
Preview splits: [b'train' b'train' b'train' b'train' b'train']
Recorded split counts: {'train': 11959, 'val': 1712, 'test': 3421}
label_texts: ['basophil', 'eosinophil', 'erythroblast', '...']
prompt_temp_for_labels: ['a microscopic image of a {} cell', 'a peripheral blood smear image of a {}', 'a bloodcell of {}']
concept_texts: ['Segmented nucleus', 'Band nucleus (band form)', 'Reniform / indented nucleus', '...']
prompt_temp_for_concepts: ['a cell photo with sign of {}', 'a photo of a cell with {}', 'a cell image indicating {}', '...']
Templates stored: ['concept_prompts_t01', 'concept_prompts_t02', 'concept_prompts_t03', 'concept_prompts_t04', 'concept_prompts_t05', 'concept_prompts_t06', 'concept_prompts_t07', 'concept_prompts_t08', 'concept_prompts_t09', 'label_prompts_t01', 'label_prompts_t02', 'label_